In [ ]:
# Google 마운트
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [32]:
# 기본 설정
import numpy as np
import cv2
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import LeakyReLU # 활성화 함수 직접 사용하려 했지만 함수 인식 못함 -> import해서 해결

In [35]:
# 데이터세트 경로 설정
dataset_path = '/content/drive/MyDrive/Dataset/Dataset/'

# 증강된 데이터세트를 저장할 경로 설정
augmented_dataset_path = '/content/drive/MyDrive/Dataset/AugmentedDataset/'

# 이미지 파일 목록과 라벨 생성
image_files = []
labels = []
for filename in os.listdir(dataset_path):
  if filename.endswith(('.png','.jpg')):    #filename 중 .png, .jpg로 끝나는 것 찾기
    image_files.append(os.path.join(dataset_path, filename))
    if 'o' in filename:    # filename = o라면 0
      labels.append(0)
    elif 'x' in filename:  # filename = x라면 1
      labels.append(1)
    else:
      labels.append(2)     # o, x 둘다 아니라면 2

# 디버깅
print(len(image_files))

360


In [36]:
# 데이터 개수 늘리기(좌우반전, 노이즈 추가 etc)
flipped_count = 0 #좌우반전 된 이미지의 개수를 세기위해 필요
for image_file in image_files:
  image_path = os.path.join(dataset_path, image_file)
  flipped_count += 1  #이미지 반전 후 카운터 1 증가

  # 이미지 읽기
  img = cv2.imread(image_path)

  # 좌우반전
  flipped_img =cv2.flip(img, 1)

  # 파일 이름 생성 (원본 파일 이름에 _flipped 추가)
  name, ext = os.path.splitext(image_file)
  flipped_image_file = f"{name}_flipped{ext}"
  flipped_image_path = os.path.join(augmented_dataset_path, flipped_image_file)

  #좌우반전 된 이미지 저장
  cv2.imwrite(flipped_image_path, flipped_img)

# 디버깅
print(flipped_count)

360


In [37]:
# 이미지 데이터세트 생성 (tf.data.Dataset 사용)
image_size = (300, 300)
BATCH_SIZE = 8 #batch size : 소그룹에 속하는 데이터
BUFFER_SIZE = tf.data.AUTOTUNE #시스템과 데이터세트에 맞춰 버퍼 크기 자동 조정

# 이미지 전처리
def preprocess_image(image_path, target_size = (300, 300)):
  img = tf.keras.preprocessing.image.load_img(image_path, target_size = target_size) #이미지 불러오기
  img = tf.keras.preprocessing.image.img_to_array(img) #이미지 배열 변환
  img = img / 255.0 #픽셀 값 정규화
  return img

# 이미지 데이터와 라벨을 Numpy 배열로 변환
images = np.array([preprocess_image(image_file) for image_file in image_files])
labels = np.array(labels)

#디버깅
print(images.shape)
print(labels.shape)
print(images[0])
print(labels[0])  #우연히 x 파일 선택?

(360, 300, 300, 3)
(360,)
[[[1. 1. 1.]
  [1. 1. 1.]
  [1. 1. 1.]
  ...
  [1. 1. 1.]
  [1. 1. 1.]
  [1. 1. 1.]]

 [[1. 1. 1.]
  [1. 1. 1.]
  [1. 1. 1.]
  ...
  [1. 1. 1.]
  [1. 1. 1.]
  [1. 1. 1.]]

 [[1. 1. 1.]
  [1. 1. 1.]
  [1. 1. 1.]
  ...
  [1. 1. 1.]
  [1. 1. 1.]
  [1. 1. 1.]]

 ...

 [[1. 1. 1.]
  [1. 1. 1.]
  [1. 1. 1.]
  ...
  [1. 1. 1.]
  [1. 1. 1.]
  [1. 1. 1.]]

 [[1. 1. 1.]
  [1. 1. 1.]
  [1. 1. 1.]
  ...
  [1. 1. 1.]
  [1. 1. 1.]
  [1. 1. 1.]]

 [[1. 1. 1.]
  [1. 1. 1.]
  [1. 1. 1.]
  ...
  [1. 1. 1.]
  [1. 1. 1.]
  [1. 1. 1.]]]
1


In [34]:
# 생성한 flipped 파일 삭제하기
augmented_dataset_path = '/content/drive/MyDrive/Dataset/Dataset/'

# augmented_dataset_path 디렉토리 내의 파일 목록 가져오기
for filename in os.listdir(augmented_dataset_path):
  if "flipped" in filename:  # 파일 이름에 "flipped"가 포함된 파일 찾기
    file_path = os.path.join(augmented_dataset_path, filename)
    os.remove(file_path)  # 파일 삭제

# 디버깅
print(flipped_count)

360


In [38]:
# 훈련 데이터세트, 테스트 데이터세트로 나누기
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(images, labels, test_size = 0.2, random_state = 42)

# 디버깅
print(x_train.shape) #훈련 데이터세트에 있는 이미지의 개수, tragetsize(300, 300), 채널(RGB)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(288, 300, 300, 3)
(72, 300, 300, 3)
(288,)
(72,)


In [ ]:
# 모델 구성1 : 기본 모델
model = keras.Sequential([
    layers.Conv2D(30, (3, 3), activation='relu', input_shape=(300, 300, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(60, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(120, activation='relu'),
    layers.Dense(3, activation='softmax')
])

# layers.Conv2D : 합성곱 레이어 -> 30 : 30개의 필터 사용(다른 숫자 사용가능) / 필터 크기 3x3 / 픽셀크기 300x300 + 3개의 채널(빨,파,초)
# layers.MaxPooling2D : 최대 풀링 레이어 -> 이미지 크기 줄이고 중요한 특징 강조
# layers.Conv2D : 합성곱 레이어 -> 60개의 필터를 사용하여 더 많은 특징 추출
# layers.MaxPooling2D : 최대 풀링 레이어 -> 이미지 크기를 더 줄임
# layers.Flatten : 플래튼 레이어 -> 다차원 데이터를 1차원으로 변환
# layers.Dense : 완전 연결 레이어 -> 모든 입력 노드가 모든 출력 노드에 연결됨(120개의 노드를 가지고 있음)
# layers.Dense : 완전 연결 레이어 -> 최종 출력 레이어 -> 3개의 클래스 분류

# 모델 컴파일
model.compile(optimizer='adam', loss = 'sparse_categorical_crossentropy', metrics=['accuracy'])
# optimizer='adam' : 학습 중 오차를 줄이기 위해 모델이 가중치를 업데이트하는 방법 제어
# sparse_categorical_crossentropy : 정수형 -> 모델의 예측과 실제 목표 값 사이 차이를 정량화하는 함수
# metrics=['accuracy'] : 학습 중 모델의 성능을 평가하는데 사용

# 모델 학습
model.fit(x_train, y_train, epochs = 50, validation_data=(x_test, y_test))
# model.fit : 저장된 모델을 학습시키는 함수
# 훈련 데이터 : x_train(이미지 데이터), y_train(o, x 구분)
# epochs : 모든 데이터셋을 학습하는 횟수
# validation_data : 모델의 성능 검증
# x_test : 검증용 이미지 데이터, y_test : 검증용 이미지에 대한 정답

# 정확도 확인
_, accuracy = model.evaluate(x_test, y_test)
print(f'accuracy: {accuracy}')
print(f'기본모델 : {accuracy*100}%')

Epoch 1/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 15s 765ms/step - accuracy: 0.5556 - loss: 7.5820 - val_accuracy: 0.5417 - val_loss: 1.7956
Epoch 2/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 93ms/step - accuracy: 0.7903 - loss: 0.6872 - val_accuracy: 0.8056 - val_loss: 0.4546
Epoch 3/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 89ms/step - accuracy: 0.9006 - loss: 0.2346 - val_accuracy: 0.8056 - val_loss: 0.4706
Epoch 4/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 91ms/step - accuracy: 0.9720 - loss: 0.0869 - val_accuracy: 0.8194 - val_loss: 0.3815
Epoch 5/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 86ms/step - accuracy: 0.9913 - loss: 0.0325 - val_accuracy: 0.8194 - val_loss: 0.3350
Epoch 6/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 95ms/step - accuracy: 1.0000 - loss: 0.0066 - val_accuracy: 0.8056 - val_loss: 0.4099
Epoch 7/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 89ms/step - accuracy: 1.0000 - loss: 0.0014 - val_accuracy: 0.8056 - val_loss: 0.4733
Epoch 8/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 91ms/step - accuracy: 1.0000 - loss: 1.9839e-04 - val_accuracy: 0.8056 - val_loss: 0

In [ ]:
# 모델 구성2 : 컨볼루션 추가
model = keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(300,300, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(256, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(512, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dense(3, activation='softmax')
])

# 모델 컴파일
model.compile(optimizer = 'adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# 모델 학습
model.fit(x_train, y_train, epochs=50, validation_data=(x_test, y_test))

# 정확도 확인
_, accuracy = model.evaluate(x_test, y_test)
print(f'accuracy: {accuracy}')
print(f'컨볼루션 추가 모델 : {accuracy*100}%')

Epoch 1/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 13s 834ms/step - accuracy: 0.4884 - loss: 1.4630 - val_accuracy: 0.5278 - val_loss: 0.6912
Epoch 2/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 121ms/step - accuracy: 0.5011 - loss: 0.7248 - val_accuracy: 0.7361 - val_loss: 0.6474
Epoch 3/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 120ms/step - accuracy: 0.6882 - loss: 0.6004 - val_accuracy: 0.7500 - val_loss: 0.5302
Epoch 4/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 114ms/step - accuracy: 0.7908 - loss: 0.4049 - val_accuracy: 0.7778 - val_loss: 0.4218
Epoch 5/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 120ms/step - accuracy: 0.8585 - loss: 0.2807 - val_accuracy: 0.8611 - val_loss: 0.2316
Epoch 6/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 128ms/step - accuracy: 0.8896 - loss: 0.2225 - val_accuracy: 0.8750 - val_loss: 0.1928
Epoch 7/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 120ms/step - accuracy: 0.9509 - loss: 0.1211 - val_accuracy: 0.9167 - val_loss: 0.1750
Epoch 8/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 126ms/step - accuracy: 0.9804 - loss: 0.0782 - val_accuracy: 0.9167 - val_loss

In [ ]:
# 모델 구성3 : elu 모델
model = keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='elu', input_shape=(300, 300, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='elu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='elu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(256, activation='elu'),
    layers.Dropout(0.5),
    layers.Dense(3, activation='softmax')
])

# 모델 컴파일
model.compile(optimizer = 'adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# 모델 학습
model.fit(x_train, y_train, epochs=50, validation_data=(x_test, y_test))

# 정확도 확인
_, accuracy = model.evaluate(x_test, y_test)
print(f'accuracy: {accuracy}')
print(f'elu 모델 : {accuracy*100}%')

Epoch 1/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 8s 451ms/step - accuracy: 0.3527 - loss: 60.2731 - val_accuracy: 0.4583 - val_loss: 13.2797
Epoch 2/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 5s 106ms/step - accuracy: 0.4583 - loss: 8.8545 - val_accuracy: 0.5417 - val_loss: 0.8720
Epoch 3/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 102ms/step - accuracy: 0.4502 - loss: 1.7165 - val_accuracy: 0.4583 - val_loss: 0.9782
Epoch 4/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 101ms/step - accuracy: 0.5398 - loss: 0.9871 - val_accuracy: 0.5417 - val_loss: 0.6914
Epoch 5/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 109ms/step - accuracy: 0.4864 - loss: 1.0953 - val_accuracy: 0.7778 - val_loss: 0.6649
Epoch 6/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 110ms/step - accuracy: 0.5680 - loss: 0.9173 - val_accuracy: 0.4583 - val_loss: 0.7001
Epoch 7/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 103ms/step - accuracy: 0.5091 - loss: 1.0004 - val_accuracy: 0.4583 - val_loss: 0.7103
Epoch 8/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 116ms/step - accuracy: 0.5165 - loss: 1.0149 - val_accuracy: 0.4583 - val_los

In [ ]:
# 모델 구성3-1 : elu 모델(배치 정규화로 정확도 높임)
model = keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='elu', input_shape=(300, 300, 3)),
    layers.BatchNormalization(), # 배치 정규화
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='elu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='elu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(256, activation='elu'),
    layers.Dropout(0.5),
    layers.Dense(3, activation='softmax')
])

# 모델 컴파일
model.compile(optimizer = 'adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# 모델 학습
model.fit(x_train, y_train, epochs=50, validation_data=(x_test, y_test))

# 정확도 확인
_, accuracy = model.evaluate(x_test, y_test)
print(f'accuracy: {accuracy}')
print(f'elu 모델(배치 정규화로 정확도 높임) : {accuracy*100}%')

Epoch 1/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 8s 310ms/step - accuracy: 0.5855 - loss: 15.0996 - val_accuracy: 0.4583 - val_loss: 24.9812
Epoch 2/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 115ms/step - accuracy: 0.9128 - loss: 2.9817 - val_accuracy: 0.4583 - val_loss: 17.1730
Epoch 3/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 115ms/step - accuracy: 0.9285 - loss: 0.9084 - val_accuracy: 0.4861 - val_loss: 28.0336
Epoch 4/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 116ms/step - accuracy: 0.9613 - loss: 0.4447 - val_accuracy: 0.5000 - val_loss: 24.9539
Epoch 5/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 116ms/step - accuracy: 0.9773 - loss: 0.3646 - val_accuracy: 0.4861 - val_loss: 35.1967
Epoch 6/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 125ms/step - accuracy: 0.9870 - loss: 0.2669 - val_accuracy: 0.5278 - val_loss: 20.0452
Epoch 7/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 127ms/step - accuracy: 0.9945 - loss: 0.0616 - val_accuracy: 0.6250 - val_loss: 6.1066
Epoch 8/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 129ms/step - accuracy: 0.9871 - loss: 0.0940 - val_accuracy: 0.5972 - va

In [ ]:
# 모델 구성4 : elu로 변경 후 컨볼루션 추가
model = keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='elu', input_shape=(300,300, 3)),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='elu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='elu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(256, (3, 3), activation='elu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(512, (3, 3), activation='elu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(512, activation='elu'),
    layers.Dense(3, activation='softmax')
])

# 모델 컴파일
model.compile(optimizer = 'adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# 모델 학습
model.fit(x_train, y_train, epochs=50, validation_data=(x_test, y_test))

# 정확도 확인
_, accuracy = model.evaluate(x_test, y_test)
print(f'accuracy: {accuracy}')
print(f'elu로 변경 후 컨볼루션 추가 모델 : {accuracy*100}%')

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 10s 335ms/step - accuracy: 0.6171 - loss: 18.3067 - val_accuracy: 0.4583 - val_loss: 28.4819
Epoch 2/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 138ms/step - accuracy: 0.8180 - loss: 2.2065 - val_accuracy: 0.4583 - val_loss: 27.2162
Epoch 3/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 139ms/step - accuracy: 0.8372 - loss: 1.3030 - val_accuracy: 0.5417 - val_loss: 3.5871
Epoch 4/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 134ms/step - accuracy: 0.9213 - loss: 0.2045 - val_accuracy: 0.5278 - val_loss: 4.1240
Epoch 5/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 133ms/step - accuracy: 0.9721 - loss: 0.0799 - val_accuracy: 0.5139 - val_loss: 3.9614
Epoch 6/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 134ms/step - accuracy: 0.9693 - loss: 0.0430 - val_accuracy: 0.5139 - val_loss: 4.2398
Epoch 7/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 134ms/step - accuracy: 0.9816 - loss: 0.0391 - val_accuracy: 0.5139 - val_loss: 3.3756
Epoch 8/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 139ms/step - accuracy: 0.9908 - loss: 0.0267 - val_accuracy: 0.5139 - val_l

In [ ]:
# 모델 구성5 : Leaky_ReLU 모델
# 사용법 : actictation=layers.LeakyReLu(alpha=0.1) ->alpha 값 조정으로 기울기 0.1, 0.2, 0,3 비교
model = keras.Sequential([
    layers.Conv2D(30, (3, 3), activation='linear', input_shape=(300, 300, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(60, (3, 3), activation=layers.LeakyReLU(alpha=0.1)),
    # tensorflow.keras.layers 모델 안에 LeakyReLU가 있어서 Python에서 사용하기 위해서 layers. 추가
    layers.MaxPooling2D((2, 2),  padding = 'same'),
    layers.Flatten(),
    layers.Dense(120, activation=layers.LeakyReLU(alpha=0.1)),
    layers.Dense(3, activation='softmax')
])

# 모델 컴파일
model.compile(optimizer = 'adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# 모델 학습
model.fit(x_train, y_train, epochs=50, validation_data=(x_test, y_test))

# 정확도 확인
_, accuracy = model.evaluate(x_test, y_test)
print(f'accuracy: {accuracy}')
print(f'Leaky_ReLU 모델 : {accuracy*100}%')


# alpha = 0.2 : 정확도 약 82% / alpha = 0.3 : 정확도 약 80%

Epoch 1/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 4s 272ms/step - accuracy: 0.5132 - loss: 62.1083 - val_accuracy: 0.5278 - val_loss: 2.9060
Epoch 2/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 83ms/step - accuracy: 0.5714 - loss: 2.2615 - val_accuracy: 0.7500 - val_loss: 0.4863
Epoch 3/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 76ms/step - accuracy: 0.8075 - loss: 0.4065 - val_accuracy: 0.6806 - val_loss: 0.6432
Epoch 4/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 76ms/step - accuracy: 0.8287 - loss: 0.3941 - val_accuracy: 0.7778 - val_loss: 0.4689
Epoch 5/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 90ms/step - accuracy: 0.8879 - loss: 0.2806 - val_accuracy: 0.7778 - val_loss: 0.4544
Epoch 6/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 84ms/step - accuracy: 0.9081 - loss: 0.2618 - val_accuracy: 0.8056 - val_loss: 0.4109
Epoch 7/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 85ms/step - accuracy: 0.9077 - loss: 0.2206 - val_accuracy: 0.7778 - val_loss: 0.4793
Epoch 8/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 85ms/step - accuracy: 0.8989 - loss: 0.2227 - val_accuracy: 0.8056 - val_loss: 0.424

In [ ]:
# 모델 구성5-3 : Leaky_ReLU 모델로 변경 후 컨볼루션 추가
model = keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='linear', input_shape=(300,300, 3)),
    layers.MaxPooling2D((2, 2), padding = 'same'),
    layers.Conv2D(64, (3, 3), activation=layers.LeakyReLU(alpha=0.1)),
    layers.MaxPooling2D((2, 2), padding = 'same'),
    layers.Conv2D(128, (3, 3), activation=layers.LeakyReLU(alpha=0.1)),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2), padding = 'same'),
    layers.Conv2D(256, (3, 3), activation=layers.LeakyReLU(alpha=0.1)),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2), padding = 'same'),
    layers.Conv2D(512, (3, 3), activation=layers.LeakyReLU(alpha=0.1)),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2), padding = 'same'),
    layers.Flatten(),
    layers.Dense(512, activation='elu'),
    layers.Dense(3, activation='softmax')
])

# 모델 컴파일
model.compile(optimizer = 'adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# 모델 학습
model.fit(x_train, y_train, epochs=50, validation_data=(x_test, y_test))

# 정확도 확인
_, accuracy = model.evaluate(x_test, y_test)
print(f'accuracy: {accuracy}')
print(f'Leaky_ReLU 모델로 변경 후 컨볼루션 추가 모델 : {accuracy*100}%')

Epoch 1/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 7s 263ms/step - accuracy: 0.5096 - loss: 12.6877 - val_accuracy: 0.5417 - val_loss: 7.5927
Epoch 2/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 127ms/step - accuracy: 0.8108 - loss: 6.4630 - val_accuracy: 0.0833 - val_loss: 63.2312
Epoch 3/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 126ms/step - accuracy: 0.8973 - loss: 1.9656 - val_accuracy: 0.0278 - val_loss: 107.6003
Epoch 4/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 118ms/step - accuracy: 0.8975 - loss: 1.1830 - val_accuracy: 0.0278 - val_loss: 87.2097
Epoch 5/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 118ms/step - accuracy: 0.9326 - loss: 0.2831 - val_accuracy: 0.0278 - val_loss: 186.5565
Epoch 6/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 120ms/step - accuracy: 0.9741 - loss: 0.0861 - val_accuracy: 0.0139 - val_loss: 257.8341
Epoch 7/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 117ms/step - accuracy: 0.9641 - loss: 0.0793 - val_accuracy: 0.0278 - val_loss: 277.7209
Epoch 8/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 123ms/step - accuracy: 0.9450 - loss: 0.1941 - val_accuracy: 0.0278 

In [45]:
# k-fold 적용 후 정확도 산출_기본 설정
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score

# 5겹 교차 검증
kfold = KFold(n_splits=5, shuffle=True, random_state=22)

# 정확도 저장 리스트 초기화
accuracy_scores = []
# k-fold 수행
for train_index, test_index in kfold.split(images, labels):
  x_train, x_test = images[train_index], images[test_index]
  y_train, y_test = labels[train_index], labels[test_index]

# 모델 구성2 : 컨볼루션 추가(정확도가 높았음)
model = keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(300,300, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(256, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(512, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dense(3, activation='softmax')
])

# 모델 컴파일
model.compile(optimizer = 'adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# 모델 학습
model.fit(x_train, y_train, epochs=50, validation_data=(x_test, y_test))


# 예측
y_pred = np.argmax(model.predict(x_test), axis=1)

# 정확도 계산 후 저장
accuracy = accuracy_score(y_test, y_pred)
accuracy_scores.append(accuracy)

# 평균 정확도 출력
print(f'accuracy : {np.mean(accuracy_scores)}')
print(f'k-fold 적용 정확도 : {np.mean(accuracy_scores)*100}%')

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 8s 475ms/step - accuracy: 0.4839 - loss: 1.2779 - val_accuracy: 0.4583 - val_loss: 0.7508
Epoch 2/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 5s 133ms/step - accuracy: 0.5989 - loss: 0.6648 - val_accuracy: 0.7222 - val_loss: 0.5433
Epoch 3/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 117ms/step - accuracy: 0.7284 - loss: 0.5215 - val_accuracy: 0.7361 - val_loss: 0.5267
Epoch 4/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 114ms/step - accuracy: 0.7959 - loss: 0.4109 - val_accuracy: 0.8611 - val_loss: 0.3185
Epoch 5/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 124ms/step - accuracy: 0.8362 - loss: 0.2901 - val_accuracy: 0.8472 - val_loss: 0.2907
Epoch 6/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 121ms/step - accuracy: 0.8936 - loss: 0.2427 - val_accuracy: 0.9167 - val_loss: 0.1913
Epoch 7/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 125ms/step - accuracy: 0.9520 - loss: 0.1222 - val_accuracy: 0.9306 - val_loss: 0.1667
Epoch 8/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 119ms/step - accuracy: 0.9530 - loss: 0.0970 - val_accuracy: 0.9167 - val_loss:

In [47]:
# k-fold 적용 후 정확도 산출_기본 설정
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score

# 5겹 교차 검증
kfold = KFold(n_splits=5, shuffle=True, random_state=22)

# 정확도 저장 리스트 초기화
accuracy_scores = []
# k-fold 수행
for train_index, test_index in kfold.split(images, labels):
  x_train, x_test = images[train_index], images[test_index]
  y_train, y_test = labels[train_index], labels[test_index]

# 모델 구성2 : 컨볼루션 추가(정확도가 높았음)
model = keras.Sequential([
    layers.Conv2D(32, (3, 3), strides=(1, 1), activation='relu', input_shape=(300,300, 3)),
    layers.MaxPooling2D((2, 2), padding='same'),
    layers.Conv2D(64, (3, 3), strides=(1, 1), activation='relu'),
    layers.MaxPooling2D((2, 2), padding='same'),
    layers.Conv2D(128, (3, 3), strides=(1, 1), activation='relu'),
    layers.MaxPooling2D((2, 2), padding='same'),
    layers.Conv2D(256, (3, 3), strides=(1, 1), activation='relu'),
    layers.MaxPooling2D((2, 2), padding='same'),
    layers.Conv2D(512, (3, 3), strides=(1, 1), activation='relu'),
    layers.MaxPooling2D((2, 2), padding='same'),
    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dense(3, activation='softmax')
])
# strides=(2, 2)로 설정시 padding=same으로 설정해서 컨볼루션 연산 불가능 -> (1, 1)로 설정

# 모델 컴파일
model.compile(optimizer = 'adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# 모델 학습
model.fit(x_train, y_train, epochs=50, validation_data=(x_test, y_test))


# 예측
y_pred = np.argmax(model.predict(x_test), axis=1)

# 정확도 계산 후 저장
accuracy = accuracy_score(y_test, y_pred)
accuracy_scores.append(accuracy)

# 평균 정확도 출력
print(f'accuracy : {np.mean(accuracy_scores)}')
print(f'k-fold / stride  적용 정확도 : {np.mean(accuracy_scores)*100}%')

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 6s 325ms/step - accuracy: 0.5418 - loss: 2.0186 - val_accuracy: 0.4583 - val_loss: 0.7858
Epoch 2/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 119ms/step - accuracy: 0.4906 - loss: 0.7223 - val_accuracy: 0.7222 - val_loss: 0.6335
Epoch 3/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 116ms/step - accuracy: 0.7510 - loss: 0.5828 - val_accuracy: 0.8056 - val_loss: 0.4496
Epoch 4/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 121ms/step - accuracy: 0.8014 - loss: 0.4349 - val_accuracy: 0.8472 - val_loss: 0.3778
Epoch 5/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 122ms/step - accuracy: 0.8077 - loss: 0.3849 - val_accuracy: 0.8611 - val_loss: 0.2769
Epoch 6/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 115ms/step - accuracy: 0.8678 - loss: 0.2964 - val_accuracy: 0.9028 - val_loss: 0.1865
Epoch 7/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 121ms/step - accuracy: 0.9151 - loss: 0.2567 - val_accuracy: 0.9167 - val_loss: 0.2095
Epoch 8/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 117ms/step - accuracy: 0.9314 - loss: 0.1477 - val_accuracy: 0.9028 - val_loss: